In [22]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
import glob
from datetime import date
from datetime import datetime
import math
import requests
from math import sin, cos, sqrt, atan2, radians

In [23]:
month = '01'
df = pd.read_csv(r'airbike\2019'+month+'.csv')
df = df[['tripduration', 'starttime', 'startstationid', 'startstationlatitude',
       'startstationlongitude', 'endstationid', 'endstationlatitude',
       'endstationlongitude', 'aqstation', 'aqdistance', 'o3']]
df = df[df['o3'] != '-']
df.head(5)

,tripduration,starttime,startstationid,startstationlatitude,startstationlongitude,endstationid,endstationlatitude,endstationlongitude,aqstation,aqdistance,o3
0,320,2019-01-01 00:00:00,3160,40.778968,-73.973747,3283,40.788221,-73.970416,CCNY,5.018988,0.011000000000000001
2,591,2019-01-01 00:00:00,3171,40.785247,-73.976673,3154,40.773142,-73.958562,CCNY,4.524800,0.011000000000000001
5,535,2019-01-01 00:00:00,3630,40.803865,-73.955931,3529,40.810792,-73.943068,CCNY,1.880613,0.011000000000000001
6,280,2019-01-01 00:00:00,3675,40.784597,-73.949685,3288,40.778301,-73.948813,CCNY,3.906804,0.011000000000000001
17,3494,2019-01-01 00:00:00,3171,40.785247,-73.976673,3164,40.777057,-73.978985,CCNY,4.524800,0.011000000000000001


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 126816 entries, 0 to 967280
Data columns (total 11 columns):
tripduration             126816 non-null int64
starttime                126816 non-null object
startstationid           126816 non-null object
startstationlatitude     126816 non-null float64
startstationlongitude    126816 non-null float64
endstationid             126816 non-null object
endstationlatitude       126816 non-null float64
endstationlongitude      126816 non-null float64
aqstation                126816 non-null object
aqdistance               126816 non-null float64
o3                       126816 non-null object
dtypes: float64(5), int64(1), object(5)
memory usage: 9.2+ MB


In [25]:
def merge_start_end_coords(lat1,lon1,lat2,lon2):
    return f'{lat1},{lon1}|{lat2},{lon2}'
df['coords'] = df.apply(lambda row: merge_start_end_coords(row['startstationlatitude'],row['startstationlongitude'],row['endstationlatitude'],row['endstationlongitude'],),axis = 1)
df.head()

,tripduration,starttime,startstationid,startstationlatitude,startstationlongitude,endstationid,endstationlatitude,endstationlongitude,aqstation,aqdistance,o3,coords
0,320,2019-01-01 00:00:00,3160,40.778968,-73.973747,3283,40.788221,-73.970416,CCNY,5.018988,0.011000000000000001,"40.77896784,-73.97374737|40.7882213,-73.97041561"
2,591,2019-01-01 00:00:00,3171,40.785247,-73.976673,3154,40.773142,-73.958562,CCNY,4.524800,0.011000000000000001,"40.78524672,-73.97667321|40.77314236,-73.95856158"
5,535,2019-01-01 00:00:00,3630,40.803865,-73.955931,3529,40.810792,-73.943068,CCNY,1.880613,0.011000000000000001,"40.8038654,-73.95593079999998|40.8107922,-73.9..."
6,280,2019-01-01 00:00:00,3675,40.784597,-73.949685,3288,40.778301,-73.948813,CCNY,3.906804,0.011000000000000001,"40.784596798848916,-73.94968539476396|40.77830..."
17,3494,2019-01-01 00:00:00,3171,40.785247,-73.976673,3164,40.777057,-73.978985,CCNY,4.524800,0.011000000000000001,"40.78524672,-73.97667321|40.7770575,-73.97898475"


In [26]:
unique_trips = df['coords'].value_counts().rename_axis('coords').reset_index(name='counts')
unique_trips.head()

,coords,counts
0,"40.7869946,-73.94164802|40.7839636,-73.9471672...",226
1,"40.77140426,-73.9535166|40.77314236,-73.95856158",208
2,"40.77536905,-73.94803392|40.77801203,-73.95407149",202
3,"40.76500525,-73.95818491|40.77140426,-73.9535166",172
4,"40.779668090073116,-73.98093044757842|40.78524...",158


In [27]:
u1 = unique_trips[0:10000]
u1.shape

(10000, 2)

In [28]:
def get_distance_map_api(lat1,long1, lat2, long2):
  #key from https://developer.mapquest.com/documentation/samples/directions/v2/route/
  r = requests.get('https://www.mapquestapi.com/directions/v2/route?key=lYrP4vF3Uk5zgTiGGuEzQGwGIVDGuy24&unit=k&from='+str(lat1)+'%2C'+str(long1)+'&to='+str(lat2)+'%2C'+str(long2)+'&outFormat=json&ambiguities=ignore&routeType=bicycle&doReverseGeocode=false&enhancedNarrative=false&avoidTimedConditions=true')
  return r.json()['route']['distance']
def get_distance(coords):
    coords = coords.split('|')
    startcoords = coords[0].split(',')
    endcoords = coords[1].split(',')
    return float(get_distance_map_api(startcoords[0],startcoords[1],endcoords[0], endcoords[1]))

In [29]:
u1['tripdistance'] = u1['coords'].apply(lambda x: get_distance(x))
u1.head()

c:\program files (x86)\python37-32\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,coords,counts,tripdistance
0,"40.7869946,-73.94164802|40.7839636,-73.9471672...",226,0.9254
1,"40.77140426,-73.9535166|40.77314236,-73.95856158",208,0.5182
2,"40.77536905,-73.94803392|40.77801203,-73.95407149",202,0.5858
3,"40.76500525,-73.95818491|40.77140426,-73.9535166",172,0.8143
4,"40.779668090073116,-73.98093044757842|40.78524...",158,0.7210


In [31]:
u1.tripdistance.value_counts()

0.0000     170
1.3245      12
2.9821      11
1.5578      11
1.5176      11
          ... 
5.0710       1
4.6253       1
10.1260      1
8.7597       1
5.3639       1
Name: tripdistance, Length: 3289, dtype: int64

In [33]:
u1.to_csv(r'coordsdistance\coordsdistance1.csv')

In [30]:
'''df_airbike = df.merge(how='left', right=unique_trips, left_on='coords', right_on='coords').drop('startcoords', axis=1).drop(['coords','counts'], axis=1).fillna("")
df_airbike.head()'''

'df_airbike = df.merge(how=\'left\', right=unique_trips, left_on=\'coords\', right_on=\'coords\').drop(\'startcoords\', axis=1).drop([\'coords\',\'counts\'], axis=1).fillna("")\ndf_airbike.head()'